In [10]:
import bs4 as bs 


'''
1. load html in code
2. tranform to soup
3. strip to only body

4. loop through and grab only:
    
5. 


Output: text file 

'''

In [2]:
file_path = '/Users/huangheqing/Downloads/Ego Is the Enemy - Notebook.html'

In [8]:
f1 = open(file_path, 'r', encoding="utf-8").read()


In [11]:
soup = bs.BeautifulSoup(f1, features='html.parser') 



In [20]:
print(soup.prettify())

<!DOCTYPE html PUBLIC
"-//W3C//DTD XHTML 1.0 Strict//EN"
"XHTML1-s.dtd" >
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/TR/1999/REC-html-in-xml">
 <head>
  <meta charset="utf-8"/>
  <style>
   .bodyContainer {
            font-family: Arial, Helvetica, sans-serif;
            text-align: center;
            padding-left: 32px;
            padding-right: 32px;
        }
        
        .notebookFor {
            font-size: 18px;
            font-weight: 700;
            text-align: center;
            color: rgb(119, 119, 119);
            margin: 24px 0px 0px;
            padding: 0px;
        }
        
        .bookTitle {
            font-size: 24px;
            font-weight: 700;
            text-align: center;
            color: #333333;
            margin-top: 22px;
            padding: 0px;
        }
        
        .authors {
            font-size: 18px;
            font-weight: 700;
            text-align: center;
            color: rgb(119, 119, 119);
            ma

In [16]:
print(soup.body)

<body>
<div class="bodyContainer">
<div class="notebookFor">
                Notebook Export
            </div>
<div class="bookTitle">
                Ego Is the Enemy
            </div>
<div class="authors">
                Holiday, Ryan
            </div>
<div class="citation">
</div>
<hr/>
<div class="sectionHeading">
    THE PAINFUL PROLOGUE
</div><div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Page xxiv · Location 185
</div>
<div class="noteText">
    We must begin by seeing ourselves and the world in a new way for the first time. Then we must fight to be different and fight to stay different—that’s the hard part. I’m not saying you should repress or crush every ounce of ego in your life—or that doing so is even possible. These are just reminders, moral stories to encourage our better impulses.
</div><div class="sectionHeading">
    INTRODUCTION
</div><div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Pa

In [34]:
print(soup.find_all('div', class_='noteHeading'))

[<div class="noteHeading">
    Highlight() - Page xxiv · Location 185
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Page 2 · Location 210
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - Page 4 · Location 238
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - Page 6 · Location 266
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - TALK, TALK, TALK &gt; Page 26 · Location 457
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - TALK, TALK, TALK &gt; Page 28 · Location 486
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - TO BE OR TO DO? &gt; Page 35 · Location 569
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - BECOME A STUDENT &gt; Page 39 · Location 610
</div>, <div 

In [27]:
s_tag = soup.span

In [28]:
s_tag.decomposed()

TypeError: 'NoneType' object is not callable

In [31]:
print(soup.body)

<body>
<div class="bodyContainer">
<div class="notebookFor">
                Notebook Export
            </div>
<div class="bookTitle">
                Ego Is the Enemy
            </div>
<div class="authors">
                Holiday, Ryan
            </div>
<div class="citation">
</div>
<hr/>
<div class="sectionHeading">
    THE PAINFUL PROLOGUE
</div><div class="noteHeading">
    Highlight() - Page xxiv · Location 185
</div>
<div class="noteText">
    We must begin by seeing ourselves and the world in a new way for the first time. Then we must fight to be different and fight to stay different—that’s the hard part. I’m not saying you should repress or crush every ounce of ego in your life—or that doing so is even possible. These are just reminders, moral stories to encourage our better impulses.
</div><div class="sectionHeading">
    INTRODUCTION
</div><div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Page 2 · Location 210
</div>
<div class="noteT